In [1]:
from bs4 import BeautifulSoup
import lxml
import pandas as pd
import requests

In [17]:

AGENT = {"User-Agent": "Edge"}
def get_game_data(pg_num: int=0, agent: dict=AGENT) -> tuple[list]:
    """
    """
    ROOT = "https://www.metacritic.com"
    SELECT_CRITERIA = "a.title, .clamp-metascore div, .clamp-userscore div"
    page_url = ROOT + "/browse/games/score/metascore/all?page=" + str(pg_num)
    allgames_page = requests.get(page_url, headers=agent)
    allgames_html = BeautifulSoup(allgames_page.text, "lxml")
    game_data = allgames_html.select(SELECT_CRITERIA)
    titles = [game.get_text() for game in game_data[::3]]
    game_urls = [ROOT + game["href"] for game in game_data[::3]]
    platform = [url.split("/")[4] for url in game_urls]
    metascores = [game.get_text() for game in game_data[1::3]]
    userscores = [game.get_text() for game in game_data[2::3]]
    return titles, game_urls, platform, metascores, userscores
titles, game_urls, platform, metascores, userscores  = get_game_data()

In [31]:
def get_genre(game_urls: list[str]) -> list[list[str]]:
    """
    """
    genres = []
    for url in game_urls:
        game_page = requests.get(url, headers=AGENT)
        game_html = BeautifulSoup(game_page.text, "lxml")
        genre_data = game_html.select("li.product_genre .data")
        genre_lst = list(set([genre.get_text() for genre in genre_data]))
        genres.append(genre_lst)
    return genres

genres = get_genre(game_urls)

In [ ]:
def get_critic_scores():
    """
    """
    return

In [33]:
data_dict = {
    "Title": titles,
    "Platform": platform,
    "Metascores": metascores,
    "Userscores": userscores,
    "Genre": genres,
}

# TODO: convert genre column of lists to sparse columns of each element
df = pd.DataFrame(data_dict)
df
#df[pd.Series([len(genre) for genre in df["Genre"]]) < 4]

,Title,Platform,Metascores,Userscores,Genre
0,The Legend of Zelda: Ocarina of Time,nintendo-64,99,9.1,"[Fantasy, Action Adventure]"
1,Tony Hawk's Pro Skater 2,playstation,98,7.5,"[Skateboarding, Sports, Alternative]"
2,Grand Theft Auto IV,playstation-3,98,7.8,"[Modern, Open-World, Action Adventure]"
3,SoulCalibur,dreamcast,98,8.4,"[Action, 3D, Fighting]"
4,Grand Theft Auto IV,xbox-360,98,8.0,"[Modern, Open-World, Action Adventure]"
...,...,...,...,...,...
95,Sid Meier's Civilization II,pc,94,8.7,"[Turn-Based, Historic, 4X, Strategy, General]"
96,Quake,pc,94,8.7,"[First-Person, Sci-Fi, Action, Shooter, Arcade]"
97,Call of Duty 4: Modern Warfare,xbox-360,94,8.5,"[First-Person, Modern, Action, Shooter, Arcade]"
98,BioShock Infinite,pc,94,8.6,"[First-Person, Sci-Fi, Action, Shooter, Arcade]"


In [ ]:
#while True:
    #pg_num = 0
    #fetch_game_urls(pg_num)